In [1]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import pandas as pd

In [2]:
# scrape data from one specific link from Amazon

# connect to website
url_one_product = 'https://www.amazon.co.uk/Baileys-Strawberries-Cream-Liqueur-Limited/dp/B07DHQ2GR2/ref=sr_1_15?crid=36T8Q10WTN7GS&keywords=bailey&qid=1682418024&sprefix=bailey%2Caps%2C111&sr=8-15'
# every computer has its unique user agent, can find it from here https://httpbin.org/get
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
page = requests.get(url_one_product, headers= headers)

# connect content from Amazon
soup1 = BeautifulSoup(page.content,'html.parser')
soup2 = BeautifulSoup(soup1.prettify(),'html.parser')

In [3]:
# scrape the data from html tag from Amazon
title = soup2.find(id="productTitle").get_text().strip()[0:]
# .strip() helps remove spaces
# [:2] take only first 3 characters
price_sympol = soup2.find('span', attrs={'class':'a-price-symbol'}).get_text().strip()
price_whole = soup2.find('span', attrs={'class':'a-price-whole'}).get_text().strip()[:2]
price_fraction = soup2.find('span', attrs={'class':'a-price-fraction'}).get_text().strip()
price = price_whole + '.'+ price_fraction

# Turn list of product and price into dataframe
data = [title, price_sympol, price]
df = pd.DataFrame (data).transpose()
df.columns = ['product_name', 'currency','price']
df

,product_name,currency,price
0,Baileys Strawberries and Cream Liqueur 70cl,£,44.92


In [4]:
# scrape data from multiple links of a product categrory from Amazon
url_multiple_product = 'https://www.amazon.co.uk/s?k=baileys&crid=1DMV5KY2X7177&sprefix=baileys%2Caps%2C99&ref=nb_sb_noss_1'
# # every computer has its unique user agent, can find it from here https://httpbin.org/get
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
page2 = requests.get(url_multiple_product, headers= headers)
soup3 = BeautifulSoup(page2.content,'html.parser')
soup4 = BeautifulSoup(soup3.prettify(),'html.parser')

In [5]:
page2

<Response [200]>

In [6]:
# looking for link of each product
# in HTML, all Link will be returned inside a tag
# scrape the data from html tag from Amazon
products_link = soup4.find_all('a',attrs = {'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
product_link_list =[]
# select links only from html tags
for i in products_link[:10]:
    link_list = list(i.get('href').split(" "))
# Find out the link for specific product
    link_list1 = ['https://www.amazon.co.uk/' + s for s in link_list]
    product_link_list.append(link_list1)
    

In [7]:
# connect to website
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

# Need to define new lists outside of the loop
title_list = []
price_list = []
currency_list = []

for [i] in product_link_list:
    page = requests.get(i, headers= headers)
    # connect html from Amazon
    soup1 = BeautifulSoup(page.content,'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
    title = soup2.find(id="productTitle").get_text().strip()[0:]
    
    # .strip() helps remove spaces
    # [:2] take only first 3 characters
    price_sympol = soup2.find('span', attrs={'class':'a-price-symbol'}).get_text().strip()
    price_whole = soup2.find('span', attrs={'class':'a-price-whole'}).get_text().strip()[:2]
    price_fraction = soup2.find('span', attrs={'class':'a-price-fraction'}).get_text().strip()
    price = price_whole + '.'+ price_fraction

    title_list.append(title)
    price_list.append(price)
    currency_list.append(price_sympol)

    product_info = pd.DataFrame(
        {'Product Name': title_list,
         'Currency':currency_list,
         'Price': price_list
        })
product_info



,Product Name,Currency,Price
0,The Ultimate Baileys Irish Cream & Chocolate H...,£,38.99
1,Baileys Irish Cream 350ml and Baileys Salted C...,£,36.95
2,Baileys Chocolate Indulgence Hamper presented ...,£,49.95
3,Baileys Indulgence Food Hamper - Gourmet Food ...,£,62.95
4,Baileys Original Irish Cream Liqueur 1L,£,19.99
5,Bailey's The Original Irish Cream Orange Truff...,£,16.49
6,"Baileys Original Irish Cream Liqueur, 70 cl",£,17.75
7,"Baileys Deliciously Light, 70 cl",£,16.50
8,Baileys Original Irish Cream Liqueur 1L,£,19.99
9,Baileys Chocolat Luxe Liqueur 50cl,£,17.21
